# Age prediction from eeg features

In [34]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
from utils import visualize, create_dataset_age, create_dataset_eeg, cv
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import seaborn as sns
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import pandas as pd

## Create dataset

In [35]:
target = 'Age'
# use average and clusters eeg features
# consider all the patients
data = create_dataset_eeg(SCORE = target,  clusters = False)
# remove the diagnosis colums
data.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)
data = data.rename(columns={'id':'ID'})

/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (50,78,80,91,92,93,94,95,105,106,107,108,109,119,120,121,123,133,134,135,137,276,291,292,295,296,297,300,301,302,305,306,307) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [36]:
test_indices = pd.read_csv('data/test_IDS.csv')
# Separate test and train set
data_test = pd.merge(data, test_indices, on='ID', how='inner')
data_train = data.loc[~data['ID'].isin(list((set(test_indices['ID']))))]
# labels and features
y_train = data_train[target]
X_train = data_train.drop([target, 'ID'], axis=1)
y_test = data_test[target]
X_test = data_test.drop([target, 'ID'], axis=1)

## Fix a baseline (mean age)

In [37]:
# Baseline
mean = np.mean(y_train)
print('mean_age = {}'.format(mean))
baseline_MSE = sum((mean - y_train)**2)/len(y_train)
print('baseline_MSE = {}'.format(baseline_MSE))

mean_age = 10.42460231787521
baseline_MSE = 12.913945692093447


In [38]:
print('Baseline test error: {}'.format(sum((mean - y_test)**2)/len(y_test)))

Baseline test error: 13.096224316164436


# Explore different regressors and feature selection procedures

## Use AVERAGE features

## ExtraTrees + SVR

In [39]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor(100))),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median'],
            'regression__C' : [0.5, 0.8, 1, 2],
            'regression__epsilon' : [0.05, 0.1, 0.5],
            'regression__kernel' : ['rbf'],
            'feat_select__estimator__n_estimators' : [100],
            'feat_select__threshold' : [-np.inf],
            'feat_select__max_features' : [10, 20, 40, 50, 60]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test R^2: {}'.format(np.mean(r2)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   52.1s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.2min finished


Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   56.5s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.3min finished


Fitting 5 folds for each of 120 candidates, totalling 600 fits


...

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   54.6s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.3min finished



Average expected test MSE: 5.9433877098208745

Average expected test R^2: 0.5326015700233582
Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.7min finished


Selected hyperparameters: {'feat_select__estimator__n_estimators': 100, 'feat_select__max_features': 60, 'feat_select__threshold': -inf, 'imputation__strategy': 'median', 'regression__C': 0.5, 'regression__epsilon': 0.5, 'regression__kernel': 'rbf'}
True test error: 6.596088454917218
Test r2: 0.4867384218024694


## Select KBEST + SVR

In [41]:
pipe = Pipeline([ ('imputation', SimpleImputer()),
                 ('scaling', StandardScaler()),
  ('feat_select', SelectKBest()),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__C' : [0.5, 0.8, 1, 1.5, 2],
            'regression__epsilon' : [0.05, 0.1, 0.5],
            'regression__kernel' : ['rbf'],
            'feat_select__score_func' : [f_regression],
            'feat_select__k' : [10, 20, 30, 50]}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:    6.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:    6.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 180 candidates, totalling 900 fits


...

[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:    7.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Average expected test MSE: 5.855652093552453

Average expected test r2: 0.5391894390954078
Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:    7.9s


Selected hyperparameters: {'feat_select__k': 50, 'feat_select__score_func': <function f_regression at 0x7f7d9d2cd840>, 'imputation__strategy': 'mean', 'regression__C': 0.5, 'regression__epsilon': 0.05, 'regression__kernel': 'rbf'}
True test error: 6.704547102296419
Test r2: 0.4782989266526554


[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:   12.2s finished


## Extra Trees + random forest

In [42]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', RandomForestRegressor())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'feat_select__estimator__n_estimators' : [100],
              'feat_select__estimator__min_samples_leaf' : [1, 5, 8],
            'regression__max_depth' : [3, 6, 12],
            'regression__min_samples_leaf' : [1, 3, 5]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:   33.4s finished
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:   32.8s finished
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:   32.6s finished
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Average expected test MSE: 6.14046664502383

Average expected test r2: 0.513452953452223
Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:   40.9s finished


Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 1, 'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'most_frequent', 'regression__max_depth': 3, 'regression__min_samples_leaf': 1}
True test error: 6.18795377313541
Test r2: 0.518496614907402


/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## Extra Trees + XGBoost

In [43]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'feat_select__estimator__n_estimators' : [100],
              'feat_select__estimator__min_samples_leaf' : [1, 2, 5, 6],
              'regression__booster__alpha' : [0, 0.05, 0.1, 0.2],
              'regression__booster__max_depth' : [3, 4, 5, 6]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# Nested CV
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen   
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   51.8s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  1.5min finished


[16:31:53] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  1.5min finished


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   50.3s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  1.5min finished


[16:37:50] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.726044977363513

Average expected test r2: 0.5471588622370983
Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed:  1.9min finished


[16:39:43] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 5, 'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'most_frequent', 'regression__booster__alpha': 0, 'regression__booster__max_depth': 4}
True test error: 5.277493782488981
Test r2: 0.5893422584852306


/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


## Select KBest + XGBoost

In [44]:
pipe = Pipeline([('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectKBest()),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'feat_select__score_func' : [f_regression],
            'feat_select__k' : [10, 20, 40, 50],
              'regression__booster__alpha' : [0, 0.05, 0.1, 0.2],
              'regression__booster__max_depth' : [3, 4, 6]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# Nested CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:   18.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[16:40:02] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:   18.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:   18.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[16:41:17] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.775588317746111

Average expected test r2: 0.5427744834797632
Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    6.5s


[16:41:40] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__k': 10, 'feat_select__score_func': <function f_regression at 0x7f7d9d2cd840>, 'imputation__strategy': 'mean', 'regression__booster__alpha': 0, 'regression__booster__max_depth': 3}
True test error: 5.628067946136992
Test r2: 0.562063023262859


[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:   23.1s finished
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


## Use CLUSTERS features

In [45]:
target = 'Age'
# use average and clusters eeg features
# consider all the patients
data = create_dataset_eeg(SCORE = target,  clusters = True)
# remove the diagnosis colums
data.drop(columns=['DX_01_Cat', 'DX_01_Sub', 'DX_01'], inplace=True)
data = data.rename(columns={'id':'ID'})
test_indices = pd.read_csv('data/test_IDS.csv')
# Separate test and train set
data_test = pd.merge(data, test_indices, on='ID', how='inner')
data_train = data.loc[~data['ID'].isin(list((set(test_indices['ID']))))]
# labels and features
y_train = data_train[target]
X_train = data_train.drop([target, 'ID'], axis=1)
y_test = data_test[target]
X_test = data_test.drop([target, 'ID'], axis=1)

/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (50,78,80,91,92,93,94,95,105,106,107,108,109,119,120,121,123,133,134,135,137,276,291,292,295,296,297,300,301,302,305,306,307) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


## Select K BEst + SVR

In [46]:
pipe = Pipeline([ ('imputation', SimpleImputer()),
                 ('scaling', StandardScaler()),
  ('feat_select', SelectKBest()),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'regression__C' : [0.5, 0.8, 1, 1.5],
            'regression__epsilon' : [0.05, 0.1, 0.5],
            'regression__kernel' : ['rbf'],
            'feat_select__score_func' : [f_regression],
            'feat_select__k' : [20, 80, 100, 120, 160]}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# Nested CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen      
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   20.4s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:   23.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   20.3s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:   23.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 180 candidates, totalling 900 fits


...

[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 877 out of 900 | elapsed:   22.8s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:   23.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Average expected test MSE: 5.876744298419283

Average expected test r2: 0.5343438849822456
Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 718 tasks      | elapsed:   23.6s


Selected hyperparameters: {'feat_select__k': 20, 'feat_select__score_func': <function f_regression at 0x7f7d9d2cd840>, 'imputation__strategy': 'mean', 'regression__C': 0.5, 'regression__epsilon': 0.05, 'regression__kernel': 'rbf'}
True test error: 5.865401651759347
Test r2: 0.5435953703288856


[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:   35.6s finished


## ExtraTrees + SVR

In [47]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor(100))),
  ('regression', SVR())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
            'regression__C' : [0.5, 0.8, 1, 1.5, 2],
            'regression__epsilon' : [0.05, 0.1, 0.3, 0.5],
            'regression__kernel' : ['rbf'],
            'feat_select__estimator__n_estimators' : [100],
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = True)

# outer CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE

model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.7min finished


Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.7min finished


...

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.7min finished



Average expected test MSE: 5.4681954733188025

Average expected test r2: 0.5683612201850008
Fitting 5 folds for each of 60 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   31.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  4.2min finished


Selected hyperparameters: {'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'most_frequent', 'regression__C': 0.5, 'regression__epsilon': 0.5, 'regression__kernel': 'rbf'}
True test error: 5.669036262052796
Test r2: 0.5588751547819011


## ExtraTrees + Random Forest

In [48]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', RandomForestRegressor())
])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median','most_frequent'],
              'feat_select__estimator__n_estimators' : [100],
              'feat_select__estimator__min_samples_leaf' : [1, 5, 8],
            'regression__max_depth' : [3, 6, 12],
            'regression__min_samples_leaf' : [1, 3, 5, 8]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# Nested CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  3.1min finished
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  3.0min finished
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting 5 folds for each of 108 candidates, totalling 540 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  3.1min finished
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.



Average expected test MSE: 5.644794142455083

Average expected test r2: 0.5532166880432975
Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:  4.5min finished


Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 5, 'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'mean', 'regression__max_depth': 3, 'regression__min_samples_leaf': 8}
True test error: 5.154545423343744
Test r2: 0.5989092419001723


/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## Extra Trees + xgboost

In [49]:
pipe = Pipeline([ ('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectFromModel(ExtraTreesRegressor())),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'feat_select__estimator__n_estimators' : [100],
              'feat_select__estimator__min_samples_leaf' : [1, 2, 5, 6, 8],
              'regression__booster__alpha' : [0, 0.05, 0.1, 0.2],
              'regression__booster__max_depth' : [3, 5, 6]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# Nested CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen    
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  5.3min finished


[17:27:28] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  5.1min finished


...

[17:43:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  5.3min finished


[17:48:33] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.241891502305278

Average expected test r2: 0.5870030573514917
Fitting 5 folds for each of 180 candidates, totalling 900 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 900 out of 900 | elapsed:  7.8min finished
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[17:56:24] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__estimator__min_samples_leaf': 5, 'feat_select__estimator__n_estimators': 100, 'imputation__strategy': 'most_frequent', 'regression__booster__alpha': 0.05, 'regression__booster__max_depth': 6}
True test error: 5.160746656786665
Test r2: 0.5984267051838382


## Select KBest + XGBoost

In [50]:
pipe = Pipeline([('imputation', SimpleImputer()), ('scaling', StandardScaler()),
  ('feat_select', SelectKBest()),
  ('regression', xgb.XGBRegressor())])
# Prepare sets of parameters for gridsearch
parameters = {'imputation__strategy' : ['mean', 'median', 'most_frequent'],
              'feat_select__score_func' : [f_regression],
            'feat_select__k' : [20, 40, 80, 100, 150],
              'regression__booster__alpha' : [0, 0.05, 0.1, 0.2],
              'regression__booster__max_depth' : [3, 4, 5, 6, 7]
}
model = GridSearchCV(estimator=pipe, param_grid=parameters, scoring = make_scorer(mean_squared_error), cv=5,
                   iid=False, n_jobs=-1, verbose = 1)

# Nested CV (model evaluation)
(estimated_test_error, r2) = cv(model, data=X_train, labels=y_train, n_splits=5, want_r2 = True)
print('\nAverage expected test MSE: {}'.format(np.mean(estimated_test_error)))
print('\nAverage expected test r2: {}'.format(np.mean(r2)))
# effective test MSE
model_fitted = model.fit(X_train, y_train)
# see what has been chosen     
print('Selected hyperparameters: {}'.format(model_fitted.best_params_) )
y_pred = model_fitted.best_estimator_.predict(X_test)
print('True test error: {}'.format(mean_squared_error(y_pred, y_test)))
print('Test r2: {}'.format(r2_score(y_true= y_test, y_pred = y_pred)))

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   31.8s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  1.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[17:58:06] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   33.0s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  1.8min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[17:59:52] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 5 folds for each of 300 candidates, totalling 1500 fits


...

[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   31.7s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  1.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


[18:05:04] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.

Average expected test MSE: 5.973130571037072

Average expected test r2: 0.524716393041615
Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   38.7s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  1.5min


[18:07:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Selected hyperparameters: {'feat_select__k': 20, 'feat_select__score_func': <function f_regression at 0x7f7d9d2cd840>, 'imputation__strategy': 'mean', 'regression__booster__alpha': 0, 'regression__booster__max_depth': 3}
True test error: 5.628532758197512
Test r2: 0.5620268548316169


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:  2.1min finished
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/camillca/anaconda3/envs/dslab/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \
